In [ ]:
# .env 파일내용
# OPENAI_API_KEY="TEST KEY"
# https://buly.kr/GvnaAF3

In [ ]:
%pip install langchain langchain-openai python-dotenv

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os
os.getenv("OPENAI_API_KEY")[:5]

'sk-pr'

In [ ]:
#  Langchain 의 핵심 컴포넌트
# PromptTemplate : LLM에 전달할 프롬프트를 생성
# ChatOpenAI : OpenAI의 챗 모델 - GPT-5, GPT-4, GPT-3.5 등
# Runnable : 공통 인터페이스  .invoke()
# StrOutputParser : 문자열 출력 파서

In [7]:
from langchain_core.prompts import PromptTemplate
template = 'What is a good name for a comapy that makes {sample}?'
prompt = PromptTemplate.from_template(template)
formatted_prompt =  prompt.format(sample='coffe')
print(formatted_prompt)

What is a good name for a comapy that makes coffe?


In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)
response = llm.invoke( ["human",'Hello, how are you?'] )
print(response)

content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 18, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C7uGF3VO5zssaWyISoiyGj2E1NFWR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--234baea2-1ad1-457f-a482-e34d75094232-0' usage_metadata={'input_tokens': 18, 'output_tokens': 29, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [9]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed_text = parser.invoke(response)
print(parsed_text)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
